# Magiske slanger

<center>
<figure style="display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;
}">
<img
    src="images/DALL·E 2023-03-18 20.01.53 - a cartoon snake with a black hoodie typing on a laptop (cleaned).png"
    alt="a cartoon snake with a black hoodie typing on a laptop"
    width="768px"
/>
<footer><small>Lagd med DALL·E 2 og justert av meg</small></footer>
</figure>
</center>

## Alt er objekter
Inkludert data, klasser, typer, funksjoner og kode

som blir pekt til av variabler (navn)



```python
a = b = 2

print(a, b)

a = "Bouvet"

print(a)
print(b)
```


## Eksempl(ar)er

<center>
    <figure>
        <img
             src="https://d38mjcf1nvhlba.cloudfront.net/media/o/1656679102/dsc-3326-1.jpg"
             alt="To pingvier på Akvariet i Bergen. Ping Kong til venstre og Ealinor Roosivelt til høyre"
             style="height: 768px;"
        />
        <footer><small>To pingvier på Akvariet i Bergen. Ping Kong til venstre og Ealinor Roosivelt til høyre
            <br>&copy; Akvariet i Bergen</small></footer>
    </figure>
</center>

Mye av syntaksen til Python har egne funksjoner / metoder assisiert med seg.
I mange tilfeller, gjør det det _mye_ lettere å lese og å forstå hva koden gjør.

Mange av disse metodene begynner og slutter med `__` (kalt "dunder").

`type` er en funksjon som kan gi tilbake typen på et objekt, eller lage en ny type (1 vs. 3 argumenter)

`__new__` er en statisk metode alle objekter har (fordi alle arver fra `object`)

Det er ingenting som hinder oss fra å endre navnene på selve metodene, så lenge den har navnet `__ini__` i `dict`-en vi gir til `type`

In [34]:
def __init__(self, name: str):
    self._name = name

In [35]:
Penguin = type("Penguin", (object,), {
    "__init__": __init__,
    "name": property(lambda self: self._name)
})

penguin = Penguin.__new__(Penguin)
Penguin.__init__(penguin, "Pingrid Alexandra")

In [ ]:
class Penguin:
    def __init__(self, name: str):
        self._name = name

    @property
    def name(self):
        return self._name
        
penguin = Penguin("Pingrid Alexandra")

print(penguin.name)

## 🪄 Tid for litt dypere (hverdags) magi


<center>
    <figure>
        <img
             src="https://images.unsplash.com/photo-1614767629805-3bbcf6e26c7d?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1760&q=80"
             alt="Foto av forskjellige terninger med forsjellig antall sider på et spillprett"
             style="height: 768px;"
        />
        <footer><small>Foto av <a href="https://unsplash.com/@nika_benedictova?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Nika Benedictova</a> på <a href="https://unsplash.com/s/photos/dice?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a></small></footer>
    </figure>
</center>
    

## 🪄 Tid for litt dypere (hverdags) magi

```python
import random
import math

class Dice:
    def __init__(self, sides: int):
        self.sides = sides

    def roll(self) -> int:
        # math.floor(random.random() * self.sides) + 1
        return random.randrange(1, self.sides + 1)
```

In [ ]:
d = Dice(10)
d.roll()

In [ ]:
import re
from collections import defaultdict
from collections.abc import Iterable
import random
import math


class Die:
    def __init__(self, sides: int):
        self.sides = sides

    def roll(self):
        return random.randrange(1, self.sides + 1)

    def __repr__(self):
        return f"d{self.sides}: {self.roll()}"

    def __rmul__(self, other: int):
        return Dice([self] * other)


class Dice(list):
    def __init__(self, dice, *rest):
        if not isinstance(dice, Iterable):
            dice = [dice]
        dice += list(rest)
        if not all(isinstance(die, Die) for die in dice):
            raise ValueError("Some items are not dice")
        super().__init__(dice)

    def roll(self):
        return sum(die.roll() for die in self)

    def __repr__(self):
        dice = defaultdict(lambda: 0)
        for die in self:  # type: Die
            dice[die.sides] += 1
        sides = list(dice.keys())
        sides.sort()
        return " + ".join(f"{dice[num_sides]}d{num_sides}" for num_sides in sides)


In [ ]:
d6 = Die(6)

d6.roll()

_3d6 = 3 *d6
_3d6

In [ ]:
import inspect
import secrets


def revile_thine_secret(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper


#@revile_thine_secret
def boring():
    secret = secrets.token_hex(64)
    return None


In [ ]:
for item in boring.__code__.co_positions():
    print(item)

In [ ]:
class Dice:
    def __init__(self, sides: int):
        self.sides = sides

    def roll(self):
        return random.choice(range(1, self.sides + 1))

    def __repr__(self):
        return f"d{self.sides}"

In [ ]:
class Dice(Dice):
    def __init__(self, sides: int):
        super().__init__(sides)
        self.num = 1

    def roll(self):
        total = 0
        for _ in range(self.num):
            total += super().roll()
        return total

    def __repr__(self):
        return str(self.num) if self.num > 1 else '' + super().__repr__()

    def __rmul__(self, other: int):
        if not isinstance(other, int):
            raise ValueError
        d = Dice(self.sides)
        d.num = other
        return d

In [ ]:
d = Dice(6)
(6 * d).roll()

In [ ]:
import re
from typing import Self


class Dice:
    def __init__(self, sides: int, num: int = 1, modifier: int = 0):
        if num < 1:
            raise ValueError("The must be at least one die")
        self.sides = sides
        self.num = num
        self.modifier = modifier

    def roll(self):
        total = 0
        for die in range(self.num):
            total += random.choice(range(1, self.sides + 1))
        total += self.modifier
        return total

    def __call__(self):
        return self.roll()

    def __repr__(self):
        representation = f"d{self.sides}"
        if self.modifier > 0:
            representation += f" + {self.modifier}"
        elif self.modifier < 0:
            representation += f" - {abs(self.modifier)}"

        if self.num > 1:
            representation = f"{self.num}{representation}"
        return representation

    _regex = re.compile(r"^(?P<number>\d+)?d(?P<sides>\d+) *((?P<sign>[-+]) *(?P<modifier>\d+)?)?$")

    @classmethod
    def parse(cls, text: str) -> Self:
        match = cls._regex.match(text)
        if match is None:
            raise ValueError("Invalid format")
        number = match["number"]
        if number is None:
            number = 1
        else:
            number = int(number)
        sides = int(match["sides"])
        modifier = int(match["modifier"])
        if match["sign"] == "-":
            modifier = -modifier
        return cls(sides, number, modifier)



In [ ]:
d = Dice(10, 2, -1)

In [ ]:
d = Dice.parse("3d12 -3")
d

In [ ]:
import re
_regex = re.compile(r"^(?P<number>\d+)?d(?P<sides>\d+) *((?P<sign>[-+]) *(?P<modifier>\d+)?)?$")

In [ ]:
match = _regex.match("d6 +2")

In [ ]:
match.groupdict()

In [ ]:
int(None)

In [ ]:
class Interesting(type(open)):
    pass

```python
import random


class Dice:
    def __init__(self, sides: int):
        self.sides = sides

    def roll():
        return random.randrange(1, self.sides + 1)
```

```python
import random


class Dice:
    def __init__(self, sides: int):
        self.sides = sides

    def roll(self):
        random.randrage(1, self.sides + 1)
        
```

In [5]:
class Dice:
    def __init__(self, sides: int):
        self.sides = sides

    def roll(self):
        random.randrage(1, self.sides + 1)

In [ ]:
def __init__(self, name: str):
    self.name = name

Penguin = type("Penguin", (object,), {
    "__init__": __init__,
})

p = Penguin("Pingu")